# Getting Started

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/all-space-missions-from-1957/Space_Corrected.csv')

In [ ]:
data

# Preprocessing

In [ ]:
data.drop([data.columns[0], data.columns[1], 'Location', 'Detail'], axis=1, inplace=True)

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data.columns = ['Company Name', 'Datum', 'Status Rocket', 'Rocket', 'Status Mission']

## Missing Values

In [ ]:
data.isnull().sum()

In [ ]:
data['Rocket'].unique()

In [ ]:
for value in data['Rocket']:
    print(type(value))

In [ ]:
data['Rocket'] = data['Rocket'].astype(str).apply(lambda x: x.replace(',', '')).astype(np.float32)

In [ ]:
data['Rocket'] = data['Rocket'].fillna(data['Rocket'].mean())

In [ ]:
data.isnull().sum()

## Encoding

In [ ]:
data

In [ ]:
def get_year_from_date(date):
    year = re.search(r'[^,]*$', date).group(0)
    year = re.search(r'^\s[^\s]*', year).group(0)
    return np.int16(year)

In [ ]:
def get_month_from_date(date):
    month = re.search(r'^[^0-9]*', date).group(0)
    month = re.search(r'\s.*$', month).group(0)
    return month.strip()
    

In [ ]:
data['Year'] = data['Datum'].apply(get_year_from_date)
data['Month'] = data['Datum'].apply(get_month_from_date)
data.drop('Datum', axis=1, inplace=True)

In [ ]:
data

In [ ]:
data['Status Mission'].unique()

In [ ]:
data['Status Mission'] = data['Status Mission'].apply(lambda x: x if x == 'Success' else 'Failure')

In [ ]:
encoder = LabelEncoder()

data['Status Mission'] = encoder.fit_transform(data['Status Mission'])

In [ ]:
data

In [ ]:
month_ordering = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
data['Status Rocket'].unique()

In [ ]:
status_ordering = ['StatusRetired', 'StatusActive']

In [ ]:
# Given some data, a column of that data, and an ordering of the values in that column,
# perform ordinal encoding on the column and return the result.

def ordinal_encode(data, column, ordering):
    return data[column].apply(lambda x: ordering.index(x))

In [ ]:
data['Month'] = ordinal_encode(data, 'Month', month_ordering)
data['Status Rocket'] = ordinal_encode(data, 'Status Rocket', status_ordering)

In [ ]:
data

In [ ]:
def onehot_encode(data, column):
    dummies = pd.get_dummies(data[column])
    data = pd.concat([data, dummies], axis=1)
    data.drop(column, axis=1, inplace=True)
    return data

In [ ]:
data = onehot_encode(data, 'Company Name')

In [ ]:
data

## Scaling

In [ ]:
y = data['Status Mission']
X = data.drop('Status Mission', axis=1)

In [ ]:
scaler = MinMaxScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
X

# Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [ ]:
y.sum() / len(y)

In [ ]:
inputs = tf.keras.Input(shape=(60,))
x = tf.keras.layers.Dense(16, activation='relu')(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]
)


batch_size=32
epochs=35

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs
)

In [ ]:
plt.figure(figsize=(14, 10))

epochs_range = range(1, epochs + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs_range, train_loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend('upper right')

plt.show()

In [ ]:
np.argmin(val_loss)

In [ ]:
model.evaluate(X_test, y_test)